# CS 4964 - Final Project




### Dataset
The dataset we will be using is from Huggingface.co. 

#p
https://huggingface.co/datasets/speech_commands


In [1]:
from speech_commands import SpeechCommands

In [3]:
speechCommands = SpeechCommands()



ValueError: Config name is missing.
Please pick one among the available configs: ['v0.01', 'v0.02']
Example of usage:
	`load_dataset('speech_commands', 'v0.01')`